In [1]:
### This idea for code is from Aladdin Persson, following this video:
### https://www.youtube.com/watch?v=OljTVUVzPpM&ab_channel=AladdinPersson
### His Github profile is https://github.com/aladdinpersson
### Check his youtube, he makes a lot of quality content :D

In [2]:
import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [3]:
# Hyperparameters, GANs are really hyperparameter dependent, which means
# even little tweaks can give vastly different results
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Andrej Karpathy says that for Adam this learning rate is the best,
# will test later other learning rates
LEARNING_RATE = 3e-4
# Can be 32, 64, 128, 256
Z_DIM = 64 
# MNIST images are (28, 28, 1), they're grayscale
IMAGE_DIM = 784
BATCH_SIZE = 32
N_EPOCHS = 300

In [4]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5, )),
])

In [5]:
# Load data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms, download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms, download=True)

In [6]:
# Create loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
# Classifier/discriminator

class Discriminator(nn.Module):
    def __init__(self, lr, image_dim):
        super(Discriminator, self).__init__()
        
        self.disc = nn.Sequential(
            nn.Linear(image_dim, 256),
            nn.LeakyReLU(0.1),
            
            nn.Linear(256, 128),
            nn.LeakyReLU(0.1),
            
            nn.Linear(128, 64),
            nn.LeakyReLU(0.1),
            
            nn.Linear(64, 1),    
            nn.Sigmoid(),
        )
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def forward(self, x):
        return self.disc(x)

In [8]:
class Generator(nn.Module):
    # Z dim is dimension of latent noise that image will take
    def __init__(self, lr, z_dim, image_dim):
        super(Generator, self).__init__()
        
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.LeakyReLU(0.1),
            
            nn.Linear(128, 256),
            nn.LeakyReLU(0.1),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.1),
            
            nn.Linear(512, image_dim),
            
            # Reason why we use tanh is because input data will be
            # normalized between -1 and 1 and tanh gives output between
            # -1 and 1
            nn.Tanh(),
        )
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        
    def forward(self, x):
        return self.gen(x)

In [9]:
# Creating both models

disc = Discriminator(LEARNING_RATE, IMAGE_DIM).to(device=DEVICE)
gen = Generator(LEARNING_RATE, Z_DIM, IMAGE_DIM).to(device=DEVICE)

loss = nn.BCELoss()

In [10]:
fixed_noise = torch.randn((BATCH_SIZE, Z_DIM)).to(device=DEVICE) 

In [11]:
fixed_noise

tensor([[ 0.8789, -0.4796, -0.1513,  ..., -0.6579, -1.6687, -1.3628],
        [ 0.1168, -0.5138, -0.3149,  ..., -1.1226, -0.0595,  0.8811],
        [ 0.2026, -0.0054, -0.6356,  ..., -0.3585,  1.3443, -0.0915],
        ...,
        [-0.2249, -0.7607,  0.7849,  ..., -0.3795,  1.9600, -1.4074],
        [-0.4690, -0.2048,  0.7049,  ..., -1.1894,  0.2237,  0.6776],
        [-0.4158,  0.5008, -0.6208,  ...,  1.4327, -0.5239,  1.3919]])

In [12]:
fixed_noise.shape

torch.Size([32, 64])

In [13]:
writer_fake = SummaryWriter(f"runs/GAN_MNIST/fake")
writer_real = SummaryWriter(f"runs/GAN_MNIST/real")
step = 0

In [14]:
for epoch in range(N_EPOCHS):
    for batch_idx, (real, _) in enumerate(tqdm(train_loader)):
        real = real.view(-1, 784).to(device=DEVICE)
        batch_size = real.shape[0]
        
        # Train Discriminator: max log(D(real)) + log(1 - D(G(z)))
        noise = torch.randn(batch_size, Z_DIM).to(device=DEVICE)
        
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = loss(disc_real, torch.ones_like(disc_real))
        
        disc_fake = disc(fake).view(-1)
        lossD_fake = loss(disc_fake, torch.zeros_like(disc_fake))
        
        lossD = (lossD_real + lossD_fake) / 2
        
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        disc.optimizer.step()
        
        preds = disc(fake).view(-1)
        lossG = loss(preds, torch.ones_like(preds))
        
        gen.zero_grad()
        lossG.backward()
        gen.optimizer.step()
        
        if batch_idx == 0:
            print(f"Epoch: [{epoch}/{N_EPOCHS}], Loss D: {lossD}, Loss G: {lossG}")
            
            with torch.no_grad():
                fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                data = real.reshape(-1, 1, 28, 28)
                
                image_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                image_grid_real = torchvision.utils.make_grid(data, normalize=True)
                
                writer_fake.add_image("MNIST Fake Images", image_grid_fake, global_step=step)
                writer_real.add_image("MNIST Real Images", image_grid_real, global_step=step)
                
                step += 1

  1%|          | 11/1875 [00:00<00:34, 53.94it/s]

Epoch: [0/300], Loss D: 0.6996756196022034, Loss G: 0.7555475831031799


  0%|          | 7/1875 [00:00<00:29, 62.29it/s]

Epoch: [1/300], Loss D: 0.5957714319229126, Loss G: 1.0931205749511719


  1%|          | 14/1875 [00:00<00:27, 67.59it/s]

...

  1%|          | 14/1875 [00:00<00:27, 68.66it/s]

Epoch: [298/300], Loss D: 0.5250856876373291, Loss G: 1.2601988315582275


  1%|          | 15/1875 [00:00<00:25, 72.45it/s]

Epoch: [299/300], Loss D: 0.4934687912464142, Loss G: 1.1936777830123901


100%|██████████| 1875/1875 [00:24<00:00, 77.78it/s]


In [15]:
# In perfect world discriminator should be 50/50 because it doesn't understand the difference between image
# from dataset and one that was generater